In [193]:
import requests
import lxml.html
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [12]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [24]:
doc = lxml.html.fromstring(website_url.content)

In [14]:
tree = lxml.html.fromstring(website_url.text)

In [25]:
tr_elements = doc.xpath('//tr')

In [26]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [27]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print '%d:"%s"'%(i,name)
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [28]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [29]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [99]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [67]:
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


In [68]:
c = df.columns[1]

In [44]:
c

'Neighbourhood\n'

In [100]:
df = df[['Postcode', 'Borough', c]]

In [101]:
df['Neighborhood'] = df[c]

In [102]:
df.drop([c],axis=1,inplace=True)

In [72]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [103]:
df['Neighborhood'].replace('Not assigned\n', np.nan,inplace=True)

In [104]:
df.at[8 , 'Neighborhood'] = "Queen's Park"

In [105]:
df.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Queen's Park
9,M8A,Not assigned,NaN


In [106]:
df.dropna(inplace=True)

In [107]:
df.head(10)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue\n
11,M1B,Scarborough,Rouge\n
12,M1B,Scarborough,Malvern\n


In [108]:
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [112]:
df.head(20)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge\n, Malvern\n"
1,M1C,Scarborough,"Highland Creek\n, Rouge Hill\n, Port Union\n"
2,M1E,Scarborough,"Guildwood\n, Morningside\n, West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village\n
6,M1K,Scarborough,"East Birchmount Park\n, Ionview\n, Kennedy Park\n"
7,M1L,Scarborough,"Clairlea\n, Golden Mile\n, Oakridge\n"
8,M1M,Scarborough,"Cliffcrest\n, Cliffside\n, Scarborough Village..."
9,M1N,Scarborough,"Birch Cliff\n, Cliffside West\n"


I had some issues trying to use BeatifulSoup so I used xlml.html. I got a prrety result I guess, and after reading a while about how to join duplicated postcodes I got the data frame listed above

In [114]:
df_tor = pd.read_csv('Geospatial_Coordinates.csv')

In [115]:
df_tor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [122]:
df = df.join(df_tor.set_index('Postal Code'), on='Postcode')

In [123]:
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge\n, Malvern\n",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek\n, Rouge Hill\n, Port Union\n",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n, Morningside\n, West Hill\n",43.763573,-79.188711
3,M1G,Scarborough,Woburn\n,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476


In [155]:
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize

In [140]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.653963, -79.387207.


In [197]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [147]:
df.loc[0, 'Neighborhood']

'Rouge\n, Malvern\n'

In [150]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge
, Malvern
 are 43.8066863, -79.1943534.


In [151]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=UNJEKN5EI55WJHUNXE2VHNWF1C0P0VRVO0YLITPFZSFLME0H&client_secret=S1PS05LY4OARAFSNIALPWSKBZ3FHPPPR4J0LOXCKGVQYFTEJ&v=20180605&ll=43.8066863,-79.1943534&radius=500&limit=100'

In [152]:
results = requests.get(url).json()

In [153]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [156]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [159]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


I'll split the original dataframe taking the data for every 5 postcodes, so I'm creating a new Data frame with 21 records only

In [167]:
df_new = df[::5]

In [179]:
df_new.shape

(21, 5)

This is the function to get the venues from foursquared for each neighborhood on our new dataframe

In [162]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [169]:
toronto_venues = getNearbyVenues(names=df_new['Neighborhood'],
                                   latitudes=df_new['Latitude'],
                                   longitudes=df_new['Longitude']
                                  )

In [170]:
print(toronto_venues.shape)
toronto_venues.head()

(458, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge\n, Malvern\n",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough Village\n,43.744734,-79.239476,McCowan Park,43.745089,-79.239336,Playground
2,"Dorset Park\n, Scarborough Town Centre\n, Wexf...",43.757410,-79.273304,Kairali,43.754768,-79.277199,Indian Restaurant
3,"Dorset Park\n, Scarborough Town Centre\n, Wexf...",43.757410,-79.273304,Kim Kim restaurant,43.753833,-79.276611,Chinese Restaurant
4,"Dorset Park\n, Scarborough Town Centre\n, Wexf...",43.757410,-79.273304,El Pulgarcito,43.754790,-79.277064,Latin American Restaurant


Grouping venues by neighborhood

In [171]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bloordale Gardens\n, Eringate\n, Markland Wood\n, Old Burnhamthorpe\n",6,6,6,6,6,6
"CFB Toronto\n, Downsview East\n",3,3,3,3,3,3
Christie\n,16,16,16,16,16,16
Davisville North\n,11,11,11,11,11,11
"Del Ray\n, Keelesdale\n, Mount Dennis\n, Silverthorn\n",4,4,4,4,4,4
"Design Exchange\n, Toronto Dominion Centre\n",100,100,100,100,100,100
"Dorset Park\n, Scarborough Town Centre\n, Wexford Heights\n",7,7,7,7,7,7
East Toronto\n,4,4,4,4,4,4
"First Canadian Place\n, Underground city\n",100,100,100,100,100,100


In [172]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 133 uniques categories.


In [173]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Rouge\n, Malvern\n",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scarborough Village\n,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Dorset Park\n, Scarborough Town Centre\n, Wexf...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Dorset Park\n, Scarborough Town Centre\n, Wexf...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Dorset Park\n, Scarborough Town Centre\n, Wexf...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [175]:
toronto_onehot.shape

(458, 134)

In [176]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Bloordale Gardens\n, Eringate\n, Markland Wood...",0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000
1,"CFB Toronto\n, Downsview East\n",0.00,0.333333,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000
2,Christie\n,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.062500,0.00,0.0625,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000
3,Davisville North\n,0.00,0.000000,0.00,0.00,0.000000,0.090909,0.000000,0.00,0.0000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000
4,"Del Ray\n, Keelesdale\n, Mount Dennis\n, Silve...",0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000
5,"Design Exchange\n, Toronto Dominion Centre\n",0.00,0.000000,0.04,0.01,0.000000,0.010000,0.000000,0.00,0.0000,...,0.02,0.010000,0.000000,0.00,0.01,0.01,0.000000,0.01,0.000000,0.000000
6,"Dorset Park\n, Scarborough Town Centre\n, Wexf...",0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.142857,0.00,0.000000,0.000000
7,East Toronto\n,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000
8,"First Canadian Place\n, Underground city\n",0.00,0.000000,0.03,0.01,0.000000,0.020000,0.000000,0.00,0.0000,...,0.02,0.010000,0.000000,0.00,0.01,0.01,0.000000,0.01,0.000000,0.000000
9,"Kingsview Village\n, Martin Grove Gardens\n, R...",0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.0000,...,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000


In [177]:
toronto_grouped.shape

(21, 134)

In [178]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bloordale Gardens
, Eringate
, Markland Wood
, Old Burnhamthorpe
----
          venue  freq
0      Pharmacy  0.17
1          Café  0.17
2    Beer Store  0.17
3  Liquor Store  0.17
4   Pizza Place  0.17


----CFB Toronto
, Downsview East
----
                             venue  freq
0                          Airport  0.33
1                             Park  0.33
2                         Bus Stop  0.33
3          New American Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00


----Christie
----
                venue  freq
0                Café  0.19
1       Grocery Store  0.19
2                Park  0.12
3               Diner  0.06
4  Italian Restaurant  0.06


----Davisville North
----
            venue  freq
0    Burger Joint  0.09
1             Gym  0.09
2    Dance Studio  0.09
3            Park  0.09
4  Clothing Store  0.09


----Del Ray
, Keelesdale
, Mount Dennis
, Silverthorn
----
                   venue  freq
0      Convenience Store  0.25
1  Check Cashing Service 

Lets sort our venues in descending order

In [180]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Lets select the top 5 venues on each neighborhood

In [182]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)


In [183]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Bloordale Gardens\n, Eringate\n, Markland Wood...",Park,Café,Pharmacy,Pizza Place,Liquor Store
1,"CFB Toronto\n, Downsview East\n",Park,Airport,Bus Stop,Deli / Bodega,Farmers Market
2,Christie\n,Grocery Store,Café,Park,Coffee Shop,Baby Store
3,Davisville North\n,Breakfast Spot,Food & Drink Shop,Clothing Store,Pizza Place,Dance Studio
4,"Del Ray\n, Keelesdale\n, Mount Dennis\n, Silve...",Convenience Store,Discount Store,Check Cashing Service,Sandwich Place,Yoga Studio


In [187]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 3, 4, 1, 1, 0],
      dtype=int32)

In [191]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



# check the last columns!
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge\n, Malvern\n",43.806686,-79.194353,2,Fast Food Restaurant,Yoga Studio,Food & Drink Shop,Comfort Food Restaurant,Concert Hall
5,M1J,Scarborough,Scarborough Village\n,43.744734,-79.239476,3,Playground,Yoga Studio,Dance Studio,Farmers Market,Electronics Store
10,M1P,Scarborough,"Dorset Park\n, Scarborough Town Centre\n, Wexf...",43.757410,-79.273304,1,Indian Restaurant,Pet Store,Vietnamese Restaurant,Light Rail Station,Latin American Restaurant
15,M1W,Scarborough,L'Amoreaux West\n,43.799525,-79.318389,1,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Gym Pool,Pharmacy
20,M2L,North York,"Silver Hills\n, York Mills\n",43.757490,-79.374714,4,Martial Arts Dojo,Cafeteria,Department Store,Fast Food Restaurant,Farmers Market


In [196]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters